## Using Python to Perform ETL

#### Importing the necessary libraries

In [1]:
import os
import numpy
import pandas as pd
from sqlalchemy import create_engine

#### Declaring and assigning connection variables for the MySQL server & databases 

In [2]:
host_name = "localhost"
host_ip = "127.0.0.1"
port = "3306"
user_id = "root"
pwd = "Password"

src_dbname = "humanresources"
dst_dbname = "humanresources_dw2"

#### Defining functions for getting data from and setting data into databases

In [3]:
def get_dataframe(user_id, pwd, host_name, db_name, sql_query):
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    dframe = pd.read_sql(sql_query, connection);
    connection.close()
    
    return dframe


def set_dataframe(user_id, pwd, host_name, db_name, df, table_name, pk_column, db_operation):
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    
    if db_operation == "insert":
        df.to_sql(table_name, con=connection, index=False, if_exists='replace')
        sqlEngine.execute(f"ALTER TABLE {table_name} ADD PRIMARY KEY ({pk_column});")
            
    elif db_operation == "update":
        df.to_sql(table_name, con=connection, index=False, if_exists='append')
    
    connection.close()

#### Creating the new data warehouse database

In [4]:
conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}"
sqlEngine = create_engine(conn_str, pool_recycle=3600)

sqlEngine.execute(f"DROP DATABASE IF EXISTS `{dst_dbname}`;")
sqlEngine.execute(f"CREATE DATABASE `{dst_dbname}`;")
sqlEngine.execute(f"USE {dst_dbname};")

### Creating and populating the dimension tables
#### Extracting data from the source database tables

In [5]:
# jobs
sql_jobs = "SELECT * FROM humanresources.jobs;"
df_jobs = get_dataframe(user_id, pwd, host_name, src_dbname, sql_jobs)
df_jobs.head(2)

,job_id,job_title,min_salary,max_salary
0,1,Public Accountant,4200.0,9000.0
1,2,Accounting Manager,8200.0,16000.0


In [6]:
# employees
sql_employees = "SELECT * FROM humanresources.employees;"
df_employees = get_dataframe(user_id, pwd, host_name, src_dbname, sql_employees)
df_employees.head(2)

,employee_id,first_name,last_name,email,phone_number,hire_date,job_id,salary,manager_id,department_id
0,100,Steven,King,steven.king@sqltutorial.org,515.123.4567,1987-06-17,4,24000.0,NaN,9
1,101,Neena,Kochhar,neena.kochhar@sqltutorial.org,515.123.4568,1989-09-21,5,17000.0,100.0,9


In [7]:
# departments
sql_departments = "SELECT * FROM humanresources.departments;"
df_departments = get_dataframe(user_id, pwd, host_name, src_dbname, sql_departments)
df_departments.head(2)

,department_id,department_name,location_id
0,1,Administration,1700
1,2,Marketing,1800


In [8]:
# locations
sql_locations = "SELECT * FROM humanresources.locations;"
df_locations = get_dataframe(user_id, pwd, host_name, src_dbname, sql_locations)
df_locations.head(2)

,location_id,street_address,postal_code,city,state_province,country_id
0,1400,2014 Jabberwocky Rd,26192,Southlake,Texas,US
1,1500,2011 Interiors Blvd,99236,South San Francisco,California,US


#### Performing necessary transformations

In [9]:
# jobs - renaming job_id to job_key
df_jobs.rename(columns={"job_id":"job_key"}, inplace=True)

df_jobs.head(2)

,job_key,job_title,min_salary,max_salary
0,1,Public Accountant,4200.0,9000.0
1,2,Accounting Manager,8200.0,16000.0


In [10]:
# employees - dropping first_name, last_name, email, phone_number, manager_id and renaming employee_id to employee_key, job_id to job_key, and department_id to department_key
drop_cols = ['first_name','last_name','email', 'phone_number', 'manager_id']
df_employees.drop(drop_cols, axis=1, inplace=True)
df_employees.rename(columns={"employee_id":"employee_key", "job_id": "job_key", "department_id": "department_key"}, inplace=True)

df_employees.head(2)

,employee_key,hire_date,job_key,salary,department_key
0,100,1987-06-17,4,24000.0,9
1,101,1989-09-21,5,17000.0,9


In [11]:
# departments - renaming department_id to department_key and location_id to location_key
df_departments.rename(columns={"department_id":"department_key", "location_id": "location_key"}, inplace=True)

df_departments.head(2)

,department_key,department_name,location_key
0,1,Administration,1700
1,2,Marketing,1800


In [12]:
# locations - dropping street_address, postal_code, city, country_id and renaming location_id to location_key
drop_cols = ['street_address','postal_code','city', 'country_id']
df_locations.drop(drop_cols, axis=1, inplace=True)
df_locations.rename(columns={"location_id":"location_key"}, inplace=True)

df_locations.head(2)

,location_key,state_province
0,1400,Texas
1,1500,California


#### Loading the transformed data frames into the new data warehouse by creating new tables

In [13]:
db_operation = "insert"

tables = [('dim_jobs', df_jobs, 'job_key'),
          ('dim_employees', df_employees, 'employee_key'),
          ('dim_departments', df_departments, 'department_key'),
          ('dim_locations', df_locations, 'location_key')]

In [14]:
for table_name, dataframe, primary_key in tables:
    set_dataframe(user_id, pwd, host_name, dst_dbname, dataframe, table_name, primary_key, db_operation)

### Creating and populating the fact table

In [15]:
df_fact = pd.merge(df_jobs, df_employees, on='job_key', how='inner')
df_fact.head(2)

,job_key,job_title,min_salary,max_salary,employee_key,hire_date,salary,department_key
0,1,Public Accountant,4200.0,9000.0,206,1994-06-07,8300.0,11
1,2,Accounting Manager,8200.0,16000.0,205,1994-06-07,12000.0,11


In [16]:
df_fact = pd.merge(df_fact, df_departments, on='department_key', how='inner')
df_fact.head(2)

,job_key,job_title,min_salary,max_salary,employee_key,hire_date,salary,department_key,department_name,location_key
0,1,Public Accountant,4200.0,9000.0,206,1994-06-07,8300.0,11,Accounting,1700
1,2,Accounting Manager,8200.0,16000.0,205,1994-06-07,12000.0,11,Accounting,1700


In [17]:
df_fact = pd.merge(df_fact, df_locations, on='location_key', how='inner')
df_fact.head(2)

,job_key,job_title,min_salary,max_salary,employee_key,hire_date,salary,department_key,department_name,location_key,state_province
0,1,Public Accountant,4200.0,9000.0,206,1994-06-07,8300.0,11,Accounting,1700,Washington
1,2,Accounting Manager,8200.0,16000.0,205,1994-06-07,12000.0,11,Accounting,1700,Washington


In [18]:
# dropping unnecessary columns from the fact table
drop_cols = ['min_salary','max_salary', 'state_province']
df_fact.drop(drop_cols, axis=1, inplace=True)

# reordering the columns
ordered_columns = ['employee_key', 'job_key','job_title','department_key','department_name','location_key',
                   'hire_date','salary']
df_fact = df_fact[ordered_columns]

df_fact.head(2)

,employee_key,job_key,job_title,department_key,department_name,location_key,hire_date,salary
0,206,1,Public Accountant,11,Accounting,1700,1994-06-07,8300.0
1,205,2,Accounting Manager,11,Accounting,1700,1994-06-07,12000.0


##### Writing the dataframe back to the database

In [19]:
table_name = "fact"
primary_key = "employee_key"
db_operation = "insert"

set_dataframe(user_id, pwd, host_name, dst_dbname, df_fact, table_name, primary_key, db_operation)

### Performing an aggregation - averaging the salaries of each department and looking at them in descending order to see which department has the highest average salary

In [66]:
sql_test = """
    SELECT employees.`department_name` AS `department_name`,
        AVG(employees.`salary`) AS `average_salary_in_department`
    FROM `{0}`.`fact` AS employees
    INNER JOIN `{0}`.`dim_departments` AS departments
    ON employees.department_key = departments.department_key
    GROUP BY employees.`department_name`
    ORDER BY average_salary_in_department DESC;
""".format(dst_dbname)

df_test = get_dataframe(user_id, pwd, host_name, src_dbname, sql_test)

In [67]:
df_test

,department_name,average_salary_in_department
0,Executive,19333.333333
1,Accounting,10150.000000
2,Public Relations,10000.000000
3,Sales,9616.666667
4,Marketing,9500.000000
5,Finance,8600.000000
6,Human Resources,6500.000000
7,Shipping,5885.714286
8,IT,5760.000000
9,Administration,4400.000000
